<a href="https://colab.research.google.com/github/Deepakshi-Singhla/Codsoft/blob/main/CreditCard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CreditCard Fraud Detection

In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('fraudTrain.csv') #By passing index 0, first column becomes the index column
df

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457416,457416,2019-07-21 16:09:56,30510856607165,"fraud_Conroy, Balistreri and Gorczany",health_fitness,320.88,Frederick,Robinson,M,1125 Ray Track,...,42.3958,-83.3772,96942,Horticultural consultant,1965-04-01,3aaff6b2ca79ab3807973332b8b365ef,1.342887e+09,42.486647,-82.825966,0.0
457417,457417,2019-07-21 16:10:02,180048185037117,"fraud_Hermiston, Pacocha and Smith",health_fitness,56.27,Mary,Wall,F,2481 Mills Lock,...,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,56a0491ef80224ef7e835bc6ffe222f7,1.342887e+09,41.277014,-75.141083,0.0
457418,457418,2019-07-21 16:10:22,3529597810116815,fraud_Botsford and Sons,home,33.34,Pamela,Curtis,F,8969 Fuentes Station Apt. 291,...,40.4679,-77.8967,1334,Archaeologist,1986-12-17,38abd0f5eeba96128387193dfbcc4b8d,1.342887e+09,39.867649,-78.424161,0.0
457419,457419,2019-07-21 16:10:31,213199865312311,fraud_Gaylord-Powlowski,home,53.37,Christopher,Miller,M,493 Fitzgerald Fields Suite 454,...,34.9298,-84.9885,74,Pensions consultant,1944-05-14,12bc01a235a05b9d861bce80daa17c73,1.342887e+09,35.460561,-84.511429,0.0


In [ ]:
df1 = df['is_fraud'].value_counts()
df1 #data is imbalanced

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [ ]:
legit = df[df['is_fraud'] == 0]
fraud = df[df['is_fraud'] == 1]


In [ ]:
#To balance the data
legit_sample = legit.sample(n=7506)
df = pd.concat([legit_sample, fraud], axis = 0)
df.shape

(15012, 22)

In [ ]:
outliear = len(fraud)/float(len(legit))
print(outliear)

0.005822355331224998


In [ ]:
df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.501200e+04,15012.000000,15012.000000,15012.00000,15012.000000,1.501200e+04,1.501200e+04,15012.000000,15012.000000,15012.000000
mean,4.047394e+17,299.345789,48433.966027,38.56443,-90.065977,9.136887e+04,1.348824e+09,38.562495,-90.067869,0.500000
std,1.286281e+18,389.503418,27074.205170,5.10157,13.978895,3.066054e+05,1.339088e+07,5.139935,13.987534,0.500017
min,6.041621e+10,1.000000,1257.000000,20.02710,-165.672300,2.300000e+01,1.325387e+09,19.095046,-166.550779,0.000000
25%,1.800365e+14,19.900000,25106.000000,34.77890,-96.790900,7.410000e+02,1.337331e+09,34.894031,-96.843184,0.000000
50%,3.521815e+15,87.520000,47470.000000,39.37160,-86.992000,2.518000e+03,1.349143e+09,39.374052,-87.173643,0.500000
75%,4.635050e+15,481.225000,71960.000000,41.81170,-80.062175,2.018175e+04,1.359825e+09,41.922551,-80.048248,1.000000
max,4.992346e+18,14849.740000,99783.000000,66.69330,-67.950300,2.906700e+06,1.371816e+09,67.510267,-67.016016,1.000000


In [ ]:
df_R = df[['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time' , 'merch_lat','merch_long', 'is_fraud']]
df_R.shape

(15012, 10)

In [ ]:
df_R['is_fraud'].value_counts()

0    7506
1    7506
Name: is_fraud, dtype: int64

In [ ]:
X = df_R.drop('is_fraud', axis=1)
Y = df_R['is_fraud']
X.shape
Y.shape

(15012,)

In [ ]:
xdata = X.values

ytarget = Y.values.ravel()


In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(xdata, ytarget, test_size=0.2, random_state= 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(xTrain,yTrain)

RandomForestClassifier()

In [ ]:
ypred1 = rfc.predict(xTest)

In [ ]:
acc = accuracy_score(yTest, ypred1)
print(acc)

0.9127539127539127


In [ ]:
#accuracy on test data
X_test_prediction_R = rfc.predict(xTest)
test_data_accuracy  = accuracy_score(X_test_prediction_R, yTest)
print(test_data_accuracy)

0.9127539127539127


Test Data


In [ ]:
df_test_R = pd.read_csv('fraudTest.csv', index_col=0)
df_test_R

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [ ]:
df_test_R = df_test_R[['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time' , 'merch_lat','merch_long', 'is_fraud']]

In [ ]:
X_TEST_R = df_test_R.drop('is_fraud', axis=1)
Y_TEST_R = df_test_R['is_fraud']

In [ ]:
print(X_TEST_R.shape)
print(Y_TEST_R.shape)

(555719, 9)
(555719,)


In [ ]:
Y_TEST_PRED_R = rfc.predict(X_TEST_R)
final_test1 =  accuracy_score(Y_TEST_PRED_R, Y_TEST_R)
print(final_test1)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.891072646427421
